In [1]:
# Install Dependencies
!pip install --quiet transformers diffusers accelerate safetensors sentencepiece streamlit pyngrok==7.0.0 pdfplumber python-docx bitsandbytes Pillow > /dev/null

In [2]:
# Imports
import os, time, tempfile, sys, io
from pathlib import Path
from textwrap import shorten
from PIL import Image
import torch

In [3]:
# Model loading helper with fallback strategy
from diffusers import DiffusionPipeline
def try_load_sdxl(device):
    """Try loading SDXL base (best quality). If fails, raise."""
    model_id = "stabilityai/stable-diffusion-xl-base-1.0"
    print("Attempting to load SDXL base model (best quality). This can take ~20-60s on GPU...")
    pipe = DiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        use_safetensors=True,
    )
    try:
        pipe.enable_xformers_memory_efficient_attention()
    except Exception as e:
        print("xformers not available or failed to enable:", e)
    return pipe
def try_load_sd2(device):
    """Fallback lightweight model (SD 2.1) for constrained runtimes."""
    model_id = "stabilityai/stable-diffusion-2-1-base"
    print("Loading Stable Diffusion 2.1 Base (fallback).")
    pipe = DiffusionPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16,
        use_safetensors=True,
    )
    try:
        pipe.enable_xformers_memory_efficient_attention()
    except Exception:
        pass
    return pipe

In [6]:
# Load pipeline
import torch
from diffusers import DiffusionPipeline
# GPU Check
def check_gpu():
    gpu = torch.cuda.is_available()
    if gpu:
        try:
            !nvidia-smi
        except:
            pass
    return gpu
GPU_AVAILABLE = check_gpu()
print("GPU available:", GPU_AVAILABLE)
device = "cuda" if GPU_AVAILABLE else "cpu"
pipe = None
loaded_model_name = None
print("🔍 Selecting image generation model...")
# Public Model
def try_load_absolute_reality():
    print("🟢 Loading AbsoluteReality v1.8.1 (public, fast, realistic)...")
    return DiffusionPipeline.from_pretrained(
        "digiplay/AbsoluteReality_v1.8.1",
        torch_dtype=torch.float16 if GPU_AVAILABLE else torch.float32,
        use_safetensors=True
    )
# Load Model
try:
    pipe = try_load_absolute_reality()
    loaded_model_name = "AbsoluteReality_v1.8.1"
except Exception as e:
    print("❌ Failed to load AbsoluteReality:", e)
    raise RuntimeError("🚫 No image generation model could be loaded.")
# Move Model To GPU
if GPU_AVAILABLE:
    try:
        pipe = pipe.to("cuda")
        print("🚀 Model moved to GPU successfully.")
    except Exception as e:
        print("⚠️ GPU move failed:", e)
        print("➡ Keeping model on CPU (slower).")
else:
    print("⚠ No GPU available — running on CPU (very slow).")
print(f"✅ Loaded model: {loaded_model_name}  |  Device: {device}")

GPU available: False
🔍 Selecting image generation model...
🟢 Loading AbsoluteReality v1.8.1 (public, fast, realistic)...


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

⚠ No GPU available — running on CPU (very slow).
✅ Loaded model: AbsoluteReality_v1.8.1  |  Device: cpu


In [7]:
# Summarization and character-extraction model
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
SUM_MODEL = "google/flan-t5-base"
print("Loading summarization model:", SUM_MODEL)
tok = AutoTokenizer.from_pretrained(SUM_MODEL)
model_summ = AutoModelForSeq2SeqLM.from_pretrained(SUM_MODEL).to(device if device=="cuda" else "cpu")
print("Summarization model loaded.")

Loading summarization model: google/flan-t5-base
Summarization model loaded.


In [8]:
# Text extraction helpers (txt, pdf, docx)
import pdfplumber, docx
def extract_text_from_uploaded(file_obj, filename):
    """file_obj: BytesIO or UploadedFile-like; filename: name with extension"""
    ext = filename.split(".")[-1].lower()
    if ext == "txt":
        data = file_obj.read()
        if isinstance(data, bytes):
            return data.decode(errors="ignore")
        return str(data)
    elif ext == "pdf":
        with pdfplumber.open(file_obj) as pdf:
            pages = []
            for p in pdf.pages:
                text = p.extract_text()
                if text:
                    pages.append(text)
            return "\n\n".join(pages)
    elif ext == "docx":
        tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".docx")
        tmp.write(file_obj.read())
        tmp.flush(); tmp.close()
        doc = docx.Document(tmp.name)
        paragraphs = [p.text for p in doc.paragraphs if p.text.strip() != ""]
        os.unlink(tmp.name)
        return "\n\n".join(paragraphs)
    else:
        return ""

In [9]:
# Summarize & extract characters
def summarize_article(text, max_length=200):
    prompt = f"Summarize the following article into 5 concise scene descriptions (short sentences). Keep each scene to one line:\n\n{text}"
    inputs = tok(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model_summ.device)
    outputs = model_summ.generate(**inputs, max_length=max_length, num_beams=4, early_stopping=True)
    summary = tok.decode(outputs[0], skip_special_tokens=True)
    return summary
def extract_characters(text):
    prompt = f"Extract the main characters from the article and give 1-2 short physical descriptors and roles (e.g., 'young woman, determined activist') as a comma-separated list:\n\n{text}"
    inputs = tok(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model_summ.device)
    outputs = model_summ.generate(**inputs, max_length=150, num_beams=3, early_stopping=True)
    chars = tok.decode(outputs[0], skip_special_tokens=True)
    return chars
# Prompt builder - create cinematic, high-detail prompts with face consistency hinting
def build_prompts_from_summary(summary_text, characters, max_frames=6):
    scenes = []
    if "\n" in summary_text:
        cand = [s.strip() for s in summary_text.split("\n") if s.strip()]
    else:
        cand = [s.strip() for s in summary_text.split(".") if s.strip()]
    for s in cand:
        if len(s) >= 20:
            scenes.append(s)
        if len(scenes) >= max_frames:
            break
    if not scenes:
        scenes = [shorten(summary_text, width=120)]
    prompts = []
    for idx, scene in enumerate(scenes):
        prompt = (
            f"Ultra-detailed photorealistic photograph. {scene}. "
            f"Include characters: {characters}. "
            "Cinematic framing, filmic lighting, shallow depth of field, ultra sharp details, 8k photographic realism, award-winning photo, realistic skin tones, natural shadows."
        )
        prompts.append(prompt)
    return prompts

In [10]:
# Image generation wrapper with safety and fallback
import os
BASE_SAVE_DIR = "/content/drive/MyDrive/Article_Image_Generator"
os.makedirs(BASE_SAVE_DIR, exist_ok=True)
from PIL import Image
def generate_images_for_prompts(prompts, article_name="article", width=1024, height=1024, steps=28, save_dir=BASE_SAVE_DIR):
    out_paths = []
    safe_save_dir = os.path.join(save_dir, article_name.replace(" ", "_"))
    os.makedirs(safe_save_dir, exist_ok=True)
    for i, p in enumerate(prompts):
        try:
            if device == "cuda":
                img = pipe(prompt=p, height=height, width=width, num_inference_steps=steps).images[0]
            else:
                print("Warning: Generating on CPU. This will be slow.")
                img = pipe(prompt=p, height=768, width=768, num_inference_steps=20).images[0]
            filename = os.path.join(safe_save_dir, f"frame_{i+1}.png")
            img.save(filename)
            out_paths.append(filename)
            print(f"Saved {filename}")
        except Exception as e:
            print("Image generation error for prompt:", e)
            try:
                img = pipe(prompt=p, height=768, width=768, num_inference_steps=18).images[0]
                filename = os.path.join(safe_save_dir, f"frame_{i+1}_fallback.png")
                img.save(filename)
                out_paths.append(filename)
            except Exception as e2:
                print("Fallback generation failed:", e2)
    return out_paths

In [32]:
# Streamlit Backend
MAIN_PY = r"""
import os
import io
import tempfile
import numpy as np
from typing import List
from PIL import Image, ImageFilter, ImageOps
import torch
# ============================================================
# CONFIG (Optimized for SPEED + Realism)
# ============================================================
BASE_SAVE_DIR = "/content/drive/MyDrive/Article_Image_Generator"
os.makedirs(BASE_SAVE_DIR, exist_ok=True)
SUM_MODEL = "google/flan-t5-base"
N_CANDIDATES = 1          # ⚡ FASTEST: only 1 image per prompt
IMG_HEIGHT = 640          # ⚡ lower res = faster
IMG_WIDTH = 640
SAMPLER_STEPS = 18        # ⚡ reduced steps (still good quality)
CFG_SCALE = 7.0           # balanced
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
STYLE = (
    "photorealistic, ultra-detailed, cinematic lighting, natural skin tones, "
    "film grain, 85mm lens, depth of field, award-winning photography"
)
NEGATIVE = (
    "lowres, blurry, deformed face, extra limbs, watermark, text, artifacts, bad anatomy"
)
# ============================================================
# TEXT EXTRACTION
# ============================================================
try: import pdfplumber
except: pdfplumber = None
try: import docx
except: docx = None
def extract_text_from_uploaded(file_obj, filename):
    ext = filename.split(".")[-1].lower()
    file_obj.seek(0)
    if ext == "txt":
        data = file_obj.read()
        return data.decode(errors="ignore") if isinstance(data, bytes) else str(data)
    if ext == "pdf" and pdfplumber:
        try:
            with pdfplumber.open(file_obj) as pdf:
                pages = [p.extract_text() or "" for p in pdf.pages]
            return "\n\n".join(pages)
        except:
            return ""
    if ext == "docx" and docx:
        tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".docx")
        tmp.write(file_obj.read()); tmp.close()
        d = docx.Document(tmp.name)
        os.unlink(tmp.name)
        return "\n".join([p.text for p in d.paragraphs])
    return ""
# ============================================================
# SUMMARIZATION
# ============================================================
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
_tok = None
_summ = None
def _ensure_summarizer():
    global _tok, _summ
    if _tok is None:
        _tok = AutoTokenizer.from_pretrained(SUM_MODEL)
    if _summ is None:
        _summ = AutoModelForSeq2SeqLM.from_pretrained(SUM_MODEL).to(DEVICE)
def summarize_article(text, max_scenes=6):
    _ensure_summarizer()
    prompt = f"Summarize into {max_scenes} cinematic scenes:\n{text}"
    inp = _tok(prompt, return_tensors="pt", truncation=True, max_length=1024).to(DEVICE)
    out = _summ.generate(inp.input_ids, max_length=256, num_beams=4)
    summary = _tok.decode(out[0], skip_special_tokens=True)
    lines = [l.strip() for l in summary.split("\n") if l.strip()]
    return "\n".join(lines[:max_scenes])
def extract_characters(text):
    _ensure_summarizer()
    prompt = "List main characters with 1–2 word physical descriptors:\n" + text
    inp = _tok(prompt, return_tensors="pt", truncation=True, max_length=1024).to(DEVICE)
    out = _summ.generate(inp.input_ids, max_length=80, num_beams=3)
    chars = _tok.decode(out[0], skip_special_tokens=True)
    return chars.replace("\n", " ").strip()
# ============================================================
# PROMPT BUILDER
# ============================================================
def build_prompts_from_summary(summary, characters, max_frames=6):
    lines = [l.strip() for l in summary.split("\n") if l.strip()][:max_frames]
    return [f"{l}. Characters: {characters}. {STYLE}" for l in lines]
# ============================================================
# MODEL LOADING (Optimized for speed)
# ============================================================
from diffusers import DiffusionPipeline
_pipe = None
def _load_model_fast():
    global _pipe
    models = [
        "Lykon/dreamshaper-xl-1.0",     # ⚡ fastest realistic
        "digiplay/AbsoluteReality_v1.8.1",
    ]
    dtype = torch.float16 if DEVICE == "cuda" else torch.float32
    for m in models:
        try:
            print("Loading →", m)
            _pipe = DiffusionPipeline.from_pretrained(m, torch_dtype=dtype)
            _pipe.to(DEVICE)
            print("Loaded:", m)
            return
        except Exception as e:
            print("FAILED:", m, "→", e)
    raise RuntimeError("No model could be loaded.")
def get_pipe():
    global _pipe
    if _pipe is None:
        _load_model_fast()
    return _pipe
# ============================================================
# SIMPLE SHARPENING
# ============================================================
def enhance(img):
    try:
        img = img.filter(ImageFilter.UnsharpMask(radius=0.8, percent=110))
    except:
        pass
    return img
# ============================================================
# IMAGE GENERATION (FAST)
# ============================================================
def generate_high_realism_images_from_prompts(prompts, article_name="article"):
    pipe = get_pipe()
    save_dir = os.path.join(BASE_SAVE_DIR, article_name.replace(" ", "_"))
    os.makedirs(save_dir, exist_ok=True)
    out_paths = []
    for i, p in enumerate(prompts):
        gen = torch.Generator(DEVICE).manual_seed(torch.randint(0, 999999999, (1,)).item())
        img = pipe(
            prompt=p,
            negative_prompt=NEGATIVE,
            height=IMG_HEIGHT,
            width=IMG_WIDTH,
            guidance_scale=CFG_SCALE,
            num_inference_steps=SAMPLER_STEPS,
            generator=gen
        ).images[0]
        img = enhance(img)
        out = os.path.join(save_dir, f"scene_{i+1}.png")
        img.save(out)
        out_paths.append(out)
    return out_paths
def generate_images_for_prompts(prompts, article_name):
    return generate_high_realism_images_from_prompts(prompts, article_name)
"""
with open("main.py", "w") as f:
    f.write(MAIN_PY)
print("⚡ main.py (FAST VERSION) created successfully!")

⚡ main.py (FAST VERSION) created successfully!


In [33]:
# Streamlit UI
STREAMLIT_APP = r'''
import streamlit as st
import os
# -------------------------------
# Custom Glassmorphism CSS
# -------------------------------
st.markdown("""
<style>
body {
    background: linear-gradient(135deg, #0f2027, #203a43, #2c5364);
    color: #ffffff !important;
}
.main, .block-container {
    background: transparent !important;
}
.block-container {
    backdrop-filter: blur(12px);
    -webkit-backdrop-filter: blur(12px);
    background: rgba(255,255,255,0.08) !important;
    padding: 35px 35px;
    border-radius: 16px;
    border: 1px solid rgba(255,255,255,0.18);
    margin-top: 30px;
}
h1, h2, h3, h4, h5 {
    color: #e8f1f2 !important;
    font-weight: 700;
    text-shadow: 0 1px 3px rgba(0,0,0,0.25);
}
.stRadio > label {
    color: white !important;
}
.stButton>button {
    background: linear-gradient(90deg, #ff7eb3, #ff758c);
    color: white !important;
    border-radius: 12px;
    padding: 0.65rem 1.3rem;
    font-size: 1rem;
    border: none;
    font-weight: 600;
    transition: 0.25s ease;
}
.stButton>button:hover {
    background: linear-gradient(90deg, #ff9eb9, #ff8aa7);
    transform: scale(1.05);
}
.stFileUploader label {
    color: #ffffff !important;
    font-size: 1.05rem !important;
}
hr {
    border: 1px solid rgba(255,255,255,0.25);
}
</style>
""", unsafe_allow_html=True)
# -------------------------------
# Title + Description
# -------------------------------
st.title("🌌 AI Article → Realistic Image Generator")
st.markdown("#### Convert entire articles into stunning cinematic images using AI 🎞️✨")
st.markdown("---")
# -------------------------------
# Import backend functions
# -------------------------------
from main import (
    extract_text_from_uploaded,
    summarize_article,
    extract_characters,
    build_prompts_from_summary,
    generate_images_for_prompts
)
# -------------------------------
# UI Mode Selection
# -------------------------------
mode = st.radio("Select Mode", ["🎯 Single Article", "📚 Batch Processing"])
# ====================================================================
# SINGLE ARTICLE MODE
# ====================================================================
if mode == "🎯 Single Article":
    file = st.file_uploader("📄 Upload an Article File", type=["txt","pdf","docx"])
    if file:
        with st.spinner("📘 Reading file..."):
            text = extract_text_from_uploaded(file, file.name)
        st.subheader("📝 Article Preview")
        st.write(text[:1000] + ("..." if len(text) > 1000 else ""))
        st.markdown("---")
        if st.button("🚀 Generate Cinematic Images"):
            with st.spinner("🧠 Understanding & summarizing article..."):
                summary = summarize_article(text)
                characters = extract_characters(text)
                prompts = build_prompts_from_summary(summary, characters)
            st.subheader("🎬 Generated Prompts")
            for i, p in enumerate(prompts):
                st.markdown(f"**Scene {i+1}:** {p}")
            st.markdown("---")
            with st.spinner("🎨 Creating high-quality realistic images..."):
                images = generate_images_for_prompts(prompts, article_name=file.name.split('.')[0])
            st.success("✨ Image Generation Complete!")
            for img in images:
                st.image(img, width=450)
# ====================================================================
# BATCH MODE
# ====================================================================
else:
    files = st.file_uploader("📄 Upload Multiple Article Files", type=["txt","pdf","docx"], accept_multiple_files=True)
    if files and st.button("🚀 Process All Files"):
        for file in files:
            st.write(f"📂 Processing **{file.name}**...")
            text = extract_text_from_uploaded(file, file.name)
            summary = summarize_article(text)
            characters = extract_characters(text)
            prompts = build_prompts_from_summary(summary, characters)
            images = generate_images_for_prompts(prompts, article_name=file.name.split('.')[0])
            st.success(f"✔ Finished {file.name}, generated {len(images)} images.")
        st.success("🎉 Batch Processing Complete! Images saved to Google Drive.")
'''
# Write Streamlit UI to app.py
with open("app.py", "w") as f:
    f.write(STREAMLIT_APP)
print("app.py created successfully with refined glass UI.")

app.py created successfully with refined glass UI.


In [34]:
# Inject the real functions into the main.py module so Streamlit can import them
import types
import sys
# Create a dynamic module named "main"
main_mod = types.ModuleType("main")
# Assign actual functions & variables from notebook
main_mod.extract_text_from_uploaded = extract_text_from_uploaded
main_mod.summarize_article = summarize_article
main_mod.extract_characters = extract_characters
main_mod.build_prompts_from_summary = build_prompts_from_summary
main_mod.generate_images_for_prompts = generate_images_for_prompts
main_mod.BASE_SAVE_DIR = BASE_SAVE_DIR
# Register the module so that `from main import *` works inside Streamlit
sys.modules["main"] = main_mod
print("✅ Successfully injected notebook functions into 'main' module.")
print("🎉 Streamlit app is now ready!")

✅ Successfully injected notebook functions into 'main' module.
🎉 Streamlit app is now ready!


In [35]:
# Streamlit App Deployment
!pip install -q streamlit pyngrok
import os
import time
import subprocess
from pyngrok import ngrok, conf
import logging
logging.getLogger("pyngrok.process").setLevel(logging.ERROR)
# Configure Ngrok Token
NGROK_AUTH_TOKEN = "2z0Oqv0tD166fELGCHwV2gLZwq1_2G2zUQRSs6C27k9vdzxwq"
conf.get_default().auth_token = NGROK_AUTH_TOKEN
# Prepare log directory
LOG_DIR = "/content/logs"
os.makedirs(LOG_DIR, exist_ok=True)
# Clean any running instance
!pkill -f streamlit 2>/dev/null || true
ngrok.kill()
print("⚙️ Starting Streamlit...")
# Launch Streamlit server
streamlit_cmd = [
    "streamlit", "run", "app.py",
    "--server.port", "8501",
    "--server.address", "0.0.0.0"
]
log_file_path = f"{LOG_DIR}/app_log.txt"
with open(log_file_path, "w") as log_file:
    process = subprocess.Popen(
        streamlit_cmd,
        stdout=log_file,
        stderr=log_file,
        text=True
    )
time.sleep(8)
# Create Ngrok tunnel
public_url = ngrok.connect(addr=8501)
print("🚀 Your Streamlit app is live at:", public_url)
print("📜 Logs at:", log_file_path)

^C
⚙️ Starting Streamlit...
🚀 Your Streamlit app is live at: NgrokTunnel: "https://d7fc55052b38.ngrok-free.app" -> "http://localhost:8501"
📜 Logs at: /content/logs/app_log.txt
